In [1]:
%load_ext autoreload
%autoreload 2



import pandas as pd
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv, find_dotenv
from urllib.parse import quote_plus
import requests
from datetime import datetime

import logging
from google.cloud import bigquery


# Define the base path 
clean_data_path = os.path.join("..", "data", "clean")

# 1. Read UCI Bank Marketing data
uci_df = pd.read_csv(os.path.join(clean_data_path, "uci_bank_marketing_cleaned.csv"))

# 2. Read Campaign Dimension data
campaign_df = pd.read_csv(os.path.join(clean_data_path, "campaign_dim_cleaned.csv"))

# 3. Read ECB Interest Rates data
ecb_df = pd.read_csv(os.path.join(clean_data_path, "ecb_interest_rates_cleaned.csv"))

# 4. Read Final Master DataFrame
final_df = pd.read_csv(os.path.join(clean_data_path, " Marketing_Campaign_final.csv"))

print("All cleaned datasets have been loaded successfully.")


All cleaned datasets have been loaded successfully.


### API Execution

In [ ]:
# This executes the script
%run "load_data.py"

In [2]:
import requests
import pandas as pd

# 1. Define the API address (The 'base' URL from your terminal)
BASE_URL = "http://127.0.0.1:8000"

# 2. Call the KPI Analytics endpoint
print("--- Testing KPI Endpoint ---")
try:
    response_kpi = requests.get(f"{BASE_URL}/kpi/conversion")
    if response_kpi.status_code == 200:
        print("Success! Data received:")
        print(response_kpi.json())
    else:
        print(f"Error {response_kpi.status_code}: {response_kpi.text}")
except Exception as e:
    print(f"Connection failed: {e}")

# 3. Call the Clients endpoint and turn it into a DataFrame
print("\n--- Testing Clients Endpoint (with filters) ---")
# We will use the 'limit' and 'job' parameters defined in the FastAPI code
params = {"limit": 10, "job": "management"} 

response_clients = requests.get(f"{BASE_URL}/clients", params=params)

if response_clients.status_code == 200:
    data = response_clients.json()
    df_api_results = pd.DataFrame(data)
    print("Top 5 clients from API:")
    display(df_api_results.head())

--- Testing KPI Endpoint ---
Connection failed: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /kpi/conversion (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002678C3F0440>: Failed to establish a new connection: [WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée'))

--- Testing Clients Endpoint (with filters) ---


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /clients?limit=10&job=management (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002678C3DD090>: Failed to establish a new connection: [WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée'))

### Part 2: Big Data System (BigQuery)

#### Step 1: Create the "Master Table" (Denormalization)

In [3]:
import pandas as pd
from google.cloud import bigquery
import os

# ==========================================
# 1. PREPARE DENORMALIZED DATA
# ==========================================

def get_denormalized_data(uci_df, campaign_df, ecb_df):
    """
    Combines normalized DataFrames into a single Wide Table for BigQuery.
    """
    print("\n--- Starting Denormalization for BigQuery ---")

    # A. Prepare Dates & IDs (Re-using your existing logic)
    ecb_df['date'] = pd.to_datetime(ecb_df['date'])
    ecb_df['economics_id'] = ecb_df['date'].dt.strftime('%Y%m%d').astype(int)
    
    uci_df['call_date'] = pd.to_datetime(dict(year=uci_df.year, month=uci_df.month_num, day=uci_df.last_contact_day))
    uci_df['economics_id'] = uci_df['call_date'].dt.strftime('%Y%m%d').astype(int)
    
    campaign_df['campaign_id'] = range(1, len(campaign_df) + 1)
    campaign_df['campaign_start_date'] = pd.to_datetime(campaign_df['campaign_start_date'])
    campaign_df['campaign_end_date'] = pd.to_datetime(campaign_df['campaign_end_date'])

    # B. Map Campaign IDs to UCI Interactions
    uci_df['campaign_id'] = 1 
    for _, row in campaign_df.iterrows():
        mask = (uci_df['call_date'] >= row['campaign_start_date']) & (uci_df['call_date'] <= row['campaign_end_date'])
        uci_df.loc[mask, 'campaign_id'] = row['campaign_id']

    # C. Execute Merges (The Denormalization)
    # Join Interactions with Campaigns
    wide_df = uci_df.merge(campaign_df, on='campaign_id', how='left', suffixes=('', '_drop'))
    
    # Join with Economics (Interest Rates)
    wide_df = wide_df.merge(ecb_df, on='economics_id', how='left', suffixes=('', '_drop'))

    # D. Clean up duplicated columns from joins
    wide_df = wide_df.loc[:, ~wide_df.columns.str.contains('_drop')]
    
    print(f"Denormalized Table Created: {wide_df.shape[1]} columns and {len(wide_df)} rows.")
    return wide_df

In [7]:
# ==========================================
# 3. EXECUTION
# ==========================================
from BigQuery_denormalized_data import get_denormalized_data, load_to_bigquery
# Assuming uci_df, campaign_df, ecb_df are already loaded from your ETL script
wide_table = get_denormalized_data(uci_df, campaign_df, ecb_df)



--- Starting Denormalization for BigQuery ---
Denormalized Table Created: 30 columns and 45211 rows.


In [6]:
wide_table.to_csv("../data/clean/denormalized_data.csv", index=False)

In [ ]:
# Run the function with your correct dataset
from BigQuery_denormalized_data import perform_clean_upload
perform_clean_upload(final_df)

In [ ]:
# --- 1. PREPARE DATA FOR PARTITIONING ---
# BigQuery requires a proper datetime object for time-based partitioning
import pandas as pd
wide_df['call_date'] = pd.to_datetime(wide_df['call_date'])

# --- 2. CONFIGURATION ---
MY_PROJECT_ID = "rncp-bank-marketing"  
MY_DATASET_ID = "bank_analytics"       
MY_TABLE_NAME = "denormalized_marketing"
MY_KEY_FILE   = "key.json.json"       

# --- 3. EXECUTION ---
# This call now triggers the updated logic with Partitioning and Clustering
result = load_to_bigquery(
    df=wide_df, 
    project_id=MY_PROJECT_ID, 
    dataset_id=MY_DATASET_ID, 
    table_name=MY_TABLE_NAME, 
    key_filename=MY_KEY_FILE
)

print(result)

## Hypothesis 5: A Machine Learning model trained on {Client + Macro + Campaign} features outperforms a model trained on {Client} features alone¶

In [11]:
ml_feature_set = pd.read_csv("../data/clean/denormalized_data.csv")
ml_feature_set

,client_age,job_category,marital_status,education_level,has_credit_default,account_balance,has_housing_loan,has_personal_loan,contact_type,last_contact_day,...,economics_id,campaign_id,campaign_name,campaign_start_date,campaign_end_date,campaign_channel,ecb_rate,date,rate_description,currency
0,58,management,married,tertiary,0,2143,1,0,not_reported,5,...,20080505,1,Term Deposit Campaign – May 2008,2008-05-01,2008-05-31,Unknown,3.00,2008-05-05,Deposit facility - date of changes (raw data) ...,EUR
1,44,technician,single,secondary,0,29,1,0,not_reported,5,...,20080505,1,Term Deposit Campaign – May 2008,2008-05-01,2008-05-31,Unknown,3.00,2008-05-05,Deposit facility - date of changes (raw data) ...,EUR
2,33,entrepreneur,married,secondary,0,2,1,1,not_reported,5,...,20080505,1,Term Deposit Campaign – May 2008,2008-05-01,2008-05-31,Unknown,3.00,2008-05-05,Deposit facility - date of changes (raw data) ...,EUR
3,47,blue-collar,married,not_reported,0,1506,1,0,not_reported,5,...,20080505,1,Term Deposit Campaign – May 2008,2008-05-01,2008-05-31,Unknown,3.00,2008-05-05,Deposit facility - date of changes (raw data) ...,EUR
4,33,not_reported,single,not_reported,0,1,0,0,not_reported,5,...,20080505,1,Term Deposit Campaign – May 2008,2008-05-01,2008-05-31,Unknown,3.00,2008-05-05,Deposit facility - date of changes (raw data) ...,EUR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,0,825,0,0,cellular,17,...,20101117,30,Term Deposit Campaign – Nov 2010,2010-11-01,2010-11-30,Unknown,0.25,2010-11-17,Deposit facility - date of changes (raw data) ...,EUR
45207,71,retired,divorced,primary,0,1729,0,0,cellular,17,...,20101117,30,Term Deposit Campaign – Nov 2010,2010-11-01,2010-11-30,Unknown,0.25,2010-11-17,Deposit facility - date of changes (raw data) ...,EUR
45208,72,retired,married,secondary,0,5715,0,0,cellular,17,...,20101117,30,Term Deposit Campaign – Nov 2010,2010-11-01,2010-11-30,Unknown,0.25,2010-11-17,Deposit facility - date of changes (raw data) ...,EUR
45209,57,blue-collar,married,secondary,0,668,0,0,telephone,17,...,20101117,30,Term Deposit Campaign – Nov 2010,2010-11-01,2010-11-30,Unknown,0.25,2010-11-17,Deposit facility - date of changes (raw data) ...,EUR


In [12]:
from machine_learning import prepare_duel_datasets

# 1. PREPROCESSING: Clean and Split
X_a, X_b, y = prepare_duel_datasets(ml_feature_set)

# 3. Verify the results
print("Success! Data is now preprocessed and ready.")

Success! Data is now preprocessed and ready.


In [13]:
from machine_learning import train_and_evaluate

f1_client = train_and_evaluate(X_a, y, "Model A (Client Only)")
f1_full = train_and_evaluate(X_b, y, "Model B (Integrated - The Grand Finale)")

print(f"\n ROI Justification: Integrated features provided a {((f1_full/f1_client)-1)*100:.2f}% lift in F1-Score!")

C:\Users\davyg\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] Le fichier spécifié est introuvable
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\davyg\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "C:\Users\davyg\anaconda3\Lib\subprocess.py", line 554, in run
    with Popen(*popenargs, **kwargs) as process:
         ~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\davyg\anaconda3\Lib\subprocess.py", line 1039, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
    ~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^


--- Model A (Client Only) Results ---
F1-Score: 0.3147
AUC-ROC:  0.6939

--- Model B (Integrated - The Grand Finale) Results ---
F1-Score: 0.5017
AUC-ROC:  0.8101

 ROI Justification: Integrated features provided a 59.42% lift in F1-Score!


In [19]:
# 1. Create the missing 'rate_bin' column
# We divide the interest rates into 3 categories: Low, Medium, and High
ml_feature_set['rate_bin'] = pd.qcut(ml_feature_set['ecb_rate'], q=3, labels=['Low', 'Medium', 'High'])

# 2. (Optional) If you also get a KeyError for 'balance_segment', create it here:
ml_feature_set['balance_segment'] = pd.qcut(ml_feature_set['account_balance'], q=3, labels=['Bronze', 'Silver', 'Gold'])

# 3. Now call your statistical tests
from machine_learning import test_stat_significance

test_stat_significance(ml_feature_set, 'rate_bin')      # Validates H1
test_stat_significance(ml_feature_set, 'contact_type')  # Validates H3
test_stat_significance(ml_feature_set, 'balance_segment') # Validates H4

--- Statistical Test: rate_bin vs Target ---
Chi-Square Statistic: 3095.78
P-Value: 0.0000e+00
✅ RESULT: Significant (p < 0.05). 
We reject the Null Hypothesis. The variables are dependent.
------------------------------
--- Statistical Test: contact_type vs Target ---
Chi-Square Statistic: 1035.71
P-Value: 1.2517e-225
✅ RESULT: Significant (p < 0.05). 
We reject the Null Hypothesis. The variables are dependent.
------------------------------
--- Statistical Test: balance_segment vs Target ---
Chi-Square Statistic: 398.98
P-Value: 2.3103e-87
✅ RESULT: Significant (p < 0.05). 
We reject the Null Hypothesis. The variables are dependent.
------------------------------


In [20]:
# Check expected frequencies
chi2, p, dof, expected = chi2_contingency(contingency_table)
if (expected < 5).any():
    print("Warning: Some cells have too few samples. Group categories together.")
else:
    print("Test is valid: All expected frequencies are > 5.")

NameError: name 'chi2_contingency' is not defined